# 重新訓練 YOLO 模型

本 Lab 將重新訓練 YOLO 模型識別車禍車輛，因此需要準備一些帶有中度或嚴重事故標籤的車輛圖片資料集。這邊可以從 `RoboFlow(https://roboflow.com/)`取得資料集，該資料集包含註解的圖片，並分成`訓練`資料集與`驗證`資料集。我們將使用這些資料來重新訓練 YOLO 模型。以下是訓練資料所需資料結構資訊:

1. 教我們的模型識別的對象 Encode classes，e.g. 0 - "中度事故"、1 - "嚴重事故"。
2. 資料集將位於目錄中的兩個子目錄，分別為 `train` 與 `valid`。每個子目錄內都包含 `images` 與 `labels` 目錄。
3. 每張圖片在 `labels` 目錄中，都有對應的註解文字檔案。註解文字檔案與圖片檔案名稱一致。
4. 資料集 `data.yaml` 檔案指向資料集，並描述其中的物件類別，該 YAML 檔將傳遞給模型進行重新訓練。

In [ ]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

import os
import requests
import zipfile
from tqdm.notebook import tqdm
from ultralytics import YOLO

接下來載入 YOLO 模型 `yolo8m.pt`

In [ ]:
# Load model
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

## 獲取訓練資料

以下提供了兩個訓練資料集，以 zip 檔案的形式提供：

1. `accident-full.zip` - 用於完全重新訓練模型。
2. `accident-sample.zip` - 用於當沒有時間完全重新訓練模型時，進行部分重新訓練模型。

在本 Lab 我們僅使用 accident-sample.zip 資料集。

In [ ]:
# Function to retrieve a specific dataset
def retrieve_dataset(dataset_type):

    # Check if the directory exists, if not, create it
    if not os.path.exists("./datasets/"):
        os.makedirs("./datasets/")

    URL = f"https://rhods-public.s3.amazonaws.com/sample-data/accident-data/accident-{dataset_type}.zip"

    # Check if the file exists, if not, download and unzip it
    if not os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Downloading file...")
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        t = tqdm(total=total_size, unit='iB', unit_scale=True)
        with open(f'./datasets/accident-{dataset_type}.zip', 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    if os.path.exists(f"./datasets/accident-{dataset_type}.zip"):
        print("Unzipping file...")
        with zipfile.ZipFile(f'./datasets/accident-{dataset_type}.zip', 'r') as zip_ref:
            zip_ref.extractall(path='./datasets/')
    print("Done!")


dataset_type = 'sample'
# dataset_type = 'full' # Use this line instead if you want to retrieve the full dataset
retrieve_dataset(dataset_type)

## 重新訓練自己的 YOLO 模型

首先要先了解什麼是 `epoch`。機器學習魔時是透過特定資料集傳遞到演算法進行訓練的。每次完整資料集通過演算法都被稱為完成一個 `epoch`，每個 `epoch` 將進一步改進模型的訓練。

在下面的程式碼中，將設置各種參數：

```python
results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)
```

* `epochs`: 由於這邊只是想驗證過程，因此只設定 1 個 epoch。
* `imgsz`: 模型需要被匯入的圖片大小(Size)。
* `batch`: 演算法將同時處理的圖片數量。數量越多，則結果越好，但消耗更多資源(e.g. RAM)。這邊由於 Lab 限制我們調整為 2 做驗證。

在訓練執行中，每個 epoch 都會顯示訓練與驗證階段的摘要：

* 第一與第二行顯示訓練階段結果。
* 第三與第四行顯示每個 epoch 的驗證階段結果。

在你的訓練運行中，每個epoch都會顯示訓練和驗證階段的摘要：第 1 和第 2 行顯示訓練階段的結果，第 3 和第 4 行顯示每個 epoch 的驗證階段的結果。

In [ ]:
# Train the model

results = model.train(data='./datasets/accident-sample/data.yaml', epochs=1, imgsz=640, batch=2)

P.S. 在標準的重新訓練後，如果對結果滿意，即可將模型匯出為 ONNX 格式。
（，您將 trainX 替換為您想使用的訓練會話）

以下指令中，可以將 `trainX` 替換為想使用的訓練 Session

```python
ObjDetOXModel = YOLO("runs/detect/trainX/weights/best.pt").export(format="onnx")
```

對於使用完整資料集進行訓練的過程，以及結果的完整描述，可以參考 Lab 說明。

現在已經重新訓練了 YOLO 模型，這時透過 `04-04-accident-recog.ipynb` 範例對模型進行測試，以驗證其可偵測車禍車輛。